# Code 5-52 to Code 5-64

In [1]:
import pandas as pd
from keras.models import Model
from tensorflow.keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")
Using TensorFlow backend.


This code is written in local and we bring the files from “google colab”. The code would have 3 parts - once a text is given the preprocessing (replacing appropriate words and regular expressions), the conversion to text to the required array format and the model inferring



Load we generated files from 3_Encode_Decode_letsgo_bidirec_collab_v2
1. Encoder and Decoder model files
2. encoder and decoder pickle files
3. Templates dictionary files
4. bus_stops pickle file for preprocessing

In [2]:
###use your folder here
dest_folder = 'collab_models/'

In [3]:
encoder_model = load_model(dest_folder + 'enc_model_collab_211_redo1')
decoder_model = load_model(dest_folder +'dec_model_collab_211_redo1')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
import pickle
import numpy as np
pkl_file = open(dest_folder + 'tokenizer_en_redo_1.pkl', 'rb')

tokenizer = pickle.load(pkl_file)


pkl_file = open(dest_folder + 'tokenizer_de_redo.pkl', 'rb')

tokenizer1 = pickle.load(pkl_file)

pkl_file = open( 'dict_templ.pkl', 'rb')

df_sents = pickle.load(pkl_file)


bus_sch = np.load("bus_stops.npz",allow_pickle=True) #make sure you use the .npz!
bus_sch1 = list(bus_sch['arr_0'])

This is a repeat of the process we followed in 2_Letsgo_preprocess.ipynb while training - replacing place name, replacing direction, number names, bus name, times and special characters. This gets applied to customer sentence and then fed into model

In [5]:
from fuzzywuzzy import fuzz 
import re
def repl_place_bus(ctext):
    #for i,row in t2.iterrows():
    #ctext = row[req_col]
    fg=0
    str1=ctext

    for j in ctext.split():
        #print (j)

        for k in bus_sch1:
            score = fuzz.ratio(j,k )
            if(score>=70):
                #print (i,j,k)
                fg=1
                break;
        #if(fg==1):
            #break;
        if(fg==1):
            fg=0
            str1 = str1.replace(j," place_name ")

    return str1



C:\Users\user\Anaconda2\envs\py36\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
direct_list = ['east','west','north','south']
def am_pm_direct_repl(ctext):
    ctext = re.sub('p.m','pm',ctext)
    ctext = re.sub('p m','pm',ctext)
    ctext = re.sub('a.m','am',ctext)
    ctext = re.sub('a m','am',ctext)

    for i in direct_list:
        ctext = re.sub(i,' direction ',ctext)
   
    return ctext

from num2words import num2words
word_num_list = []
word_num_th_list = []
for i in range(0,100):
    word_num_list.append(num2words(i))
    word_num_th_list.append(num2words(i,to='ordinal'))
    
def repl_num_words(ctext):
    for num,i in enumerate(word_num_th_list):
        ctext = re.sub(i,"num_th",ctext)
        ctext = re.sub(word_num_list[num],str(num),ctext)
    return ctext


def preprocces(ctext):
    ctext = ctext.lower()
    ctext1 = repl_place_bus(ctext)
    ctext1 = am_pm_direct_repl(ctext1)
    ctext1 = repl_num_words(ctext1)


    ctext1 = re.sub('[0-9]{1,2}[a-z]{1,2}','bus_name',ctext1)
    ctext1 = re.sub('[0-9]{1,2}[\.\s]*[0-9]{1,2}[\s]*pm','time_name',ctext1)
    ctext1 = re.sub('[0-9]{1,2}[\.\s]*[0-9]{1,2}[\s]*am','time_name',ctext1)

    ctext1 =re.sub('(\.\s){1,3}',' ',ctext1)
    ctext1 =re.sub('(\.){1,3}',' ',ctext1)
    
    return ctext1

Examples of preprocess functions

In [7]:
#sent = "when is the next bus to crafton boulevard 9 00 P.M east. . 85c"
sent = "54c"
preprocces(sent)

'bus_name'

We now set to convert the preprocessed into the right array of numbers as defined during training using tokenizer objects. We set padding length and number of classes for one-hot from the values in training process

In [8]:
max_len1 = 28
num_encoder_tokens =733
num_decoder_tokens = 1506
shp = 733

In [9]:
def conv_sent(sent,shp):
    sent = preprocces(sent)
    #print (sent)
    en_col_tr = [sent.split()]
    
    en_tr1= tokenizer.texts_to_sequences(en_col_tr )
    
    en_tr2 = pad_sequences(en_tr1, maxlen=max_len1, dtype='int32', padding='post')
    en_tr3 = to_categorical(en_tr2,num_classes =shp)
    
    return en_tr3


We check using our test preprocessed sentence and see if the shape is as expected (rows* length of sequence * one-hot)


In [10]:
sent1 = 'when is the next bus to crafton  place_name  time_name  direction  bus_name'
sent1_conv = conv_sent(sent1,num_encoder_tokens)
sent1_conv.shape

(1, 28, 733)

We are going to apply the model to the converted sentences.The first word will be initialised to ‘start’ token and states of the decoder to be initialized to encoder states. With that we get the next word. Now with the next word and the states of the word as the set of inputs we get the word at t+1. We repeat the process till we reach the ‘end’ token. Remember we trained on bot sentence templates and not on the actual sentences. Hence we get the template id as the output. We then convert the template id to the required word using "rev_dict" function 

In [11]:
def rev_dict(req_word):
    str1=""
    for k,v in df_sents.items():
        if(df_sents[k]==[req_word]):
            str1 = k
    return str1


In [12]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

def get_op(text1,shp):

    st_pos = 0
    input_seq = conv_sent(text1, shp)
    #print (input_seq)
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    str_all = ""
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0, st_pos] = 1
    for i in range(0,10):
        decoded_sentence = ''
        output_tokens, h, c = decoder_model.predict(
                [target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
           
        sampled_char = tokenizer1.sequences_to_texts([[sampled_token_index]])
        str1 = rev_dict(sampled_char[0])
        str_all = str_all + "\n" + str1
        
        if(sampled_char[0]=='end'):
            break;
        else:
            decoded_sentence += sampled_char[0]
            target_seq = np.zeros((1, 1, num_decoder_tokens))
            target_seq[0, 0, sampled_token_index] = 1.
            ####initializing state values
            states_value = [h, c]
            
    return str_all

Outputs from the bot

In [13]:
text1 = "54c"
get_op(text1,num_encoder_tokens)

'\n  placename  going to  placename  transportation center   \n short_num  short_num n short_num where are you leaving from  \n'

In [14]:
text1 = "next bus to boulevard"
get_op(text1,shp)

'\n did i get that  placename okaywhere would you like to leave from  \n if you  placename  to leave from  placename  and bausman say yes or press  short_num  otherwise say no or press  short_num   \n'

In [15]:
text1 = "Do you have bus to downtown in the morning"
get_op(text1,shp)

'\n when would you like to travel  \n'

We seem to have a start here. Following can be done to improve the responses

1.	We need to replace the answers with the appropriate place names and present a more meaningful answer. 
2.	We have trained on a single input. But its a conversation so we need to train it on the n-1 response and the question to get the next response
3.	Some of the transcription were not accurate, if it can be weeded out and model trained with a cleaner corpus
4.	Using embeddings (pre-trained or trained during the model) as input to LSTMs
5.	Tuning hyperparameters like learning rates or learning algorithms
6.	Stacked LSTM layers (First set of LSTMs provide inputs to the second set of LSTMs)
7.	Using Bidirectional LSTMs. Lets see an example of bidirectional lstm
8.	Advanced architectures like ‘BERT’
